# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [435]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Ask for parameters

In [436]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [437]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $t$: Time period, $t = 1, 2, ..., 12$
*  $f$: Food product,  $f = 1, 2$
*  $b$: Beverage product, $b = 1, 2, 3$

## Parameters:

*  $FC_{t,f}$: Fixed cost of producing one unit of food $f$ in period $t$ 
*  $FC_{t,b}$: Fixed cost of producing one unit of beverage $b$ in period $t$ 
*  $VC_{t,f}$: Variable cost of producing one unit of food  $f$ in period $t$
*  $VC_{t,b}$: Variable cost of producing one liter of beverage $b$ in period $t$ 
*  $SC_{t,f}$: Variable cost of storing one unit of food $f$ in period $t$ 
*  $SC_{t,b}$: Variable cost of storing one liter of beverage $b$ in period $t$ 
*  $D_{t,f}$: Demand for food $f$ in period $t$ 
*  $D_{t,b}$: Demand for beverage $b$ in period $t$ (in liters)
*  $R_{t,f}$: Revenue for selling one unit of food $f$ in period $t$
*  $R_{t,b}$: Revenue for selling one liter of beverage $b$ in period $t$
*  $C_{f}$: Storage capacity for food $f$ (in units) 
*  $C_{b}$: Storage capacity for beverage $b$ (in liters)
*  $P_{f}$: Production capacity for food $f$ (in units) 
*  $P_{b}$: Production capacity for beverage $b$ (in liters) 

## Decision Variables:

*  $X_{t,f}$: Quantity of food  $f$ produced in period $t$ (in units, integer)
*  $X_{t,b}$: Quantity of beverage $b$ produced in period $t$ (in liters, continuous)
*  $S_{t,f}$: Quantity of food $f$ stored at the end of period $t$ (in units, integer)
*  $S_{t,b}$: Quantity of beverage $b$ stored at the end of period $t$ (in liters, continuous) 
*  $Sold_{t,f}$: Quantity of food $f$ sold in period $t$ (in units, integer)
*  $Sold_{t,b}$: Quantity of beverage $b$ sold in period $t$ (in liters, continuous) 


# 2. Ask for objective

In [438]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [439]:
Markdown(response2.text)

## Objective Function:

Maximize the total profit over all time periods:

$\sum_{t=1}^{12} \left( \sum_{f=1}^{2} (R_{t,f} \cdot Sold_{t,f} - FC_{t,f} \cdot X_{t,f}  - VC_{t,f} \cdot X_{t,f} - SC_{t,f} \cdot S_{t,f})  + \sum_{b=1}^{3} (R_{t,b} \cdot Sold_{t,b} - FC_{t,b} \cdot X_{t,b} - VC_{t,b} \cdot X_{t,b} - SC_{t,b} \cdot S_{t,b}) \right)$ 


# 3. Ask for constraints

In [440]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [441]:
Markdown(response3.text)

## Constraints:

**1. Production Capacity:**

*   $X_{t,f} \leq P_f$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $X_{t,b} \leq P_b$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$

**2. Storage Capacity:**

*   $S_{t,f} \leq C_f$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $S_{t,b} \leq C_b$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$

**3. Demand Fulfillment:**

*   $Sold_{t,f} \leq D_{t,f}$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $Sold_{t,b} \leq D_{t,b}$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$

**4. Inventory Balance:**

*   $S_{t,f} = S_{t-1,f} + X_{t,f} - Sold_{t,f}$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$ 
*   $S_{t,b} = S_{t-1,b} + X_{t,b} - Sold_{t,b}$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$
*   $S_{0,f} = 0$ for all $f = 1, 2$ (assuming no initial inventory)
*   $S_{0,b} = 0$ for all $b = 1, 2, 3$ (assuming no initial inventory)

**5. Non-Negativity:**

*   $X_{t,f} \geq 0$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $X_{t,b} \geq 0$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$
*   $S_{t,f} \geq 0$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $S_{t,b} \geq 0$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$
*   $Sold_{t,f} \geq 0$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $Sold_{t,b} \geq 0$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$ 

**6. Integer Constraints:**

*   $X_{t,f}, S_{t,f}, Sold_{t,f} \in \mathbb{Z}$ (integers) for all $t = 1, 2, ..., 12$ and $f = 1, 2$ 


## 3. Generate the pyomo code

In [442]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [443]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
time_periods = [t+1 for t in range(12)]  
food_products = [1, 2]
beverage_products = [1, 2, 3]

# Parameters (Sample Data)
FC = {(1, 1): 10, (1, 2): 12, (2, 1): 11, (2, 2): 13} # etc. for all t, f
VC = {(1, 1): 5, (1, 2): 6, (2, 1): 5.5, (2, 2): 6.5} # etc. for all t, f
SC = {(1, 1): 2, (1, 2): 2.5, (2, 1): 2.2, (2, 2): 2.7} # etc. for all t, f
D = {(1, 1): 100, (1, 2): 120, (2, 1): 110, (2, 2): 130} # etc. for all t, f
R = {(1, 1): 20, (1, 2): 22, (2, 1): 21, (2, 2): 23} # etc. for all t, f
C = {1: 200, 2: 250}  
P = {1: 150, 2: 180} 

# Create a Concrete Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=time_periods)
model.F = pyo.Set(initialize=food_products)
model.B = pyo.Set(initialize=beverage_products)

# Parameters
model.FC_food = pyo.Param(model.T, model.F, initialize=FC, default=0)
model.FC_beverage = pyo.Param(model.T, model.B, initialize=FC, default=0)
model.VC_food = pyo.Param(model.T, model.F, initialize=VC, default=0)
model.VC_beverage = pyo.Param(model.T, model.B, initialize=VC, default=0)
model.SC_food = pyo.Param(model.T, model.F, initialize=SC, default=0)
model.SC_beverage = pyo.Param(model.T, model.B, initialize=SC, default=0)
model.D_food = pyo.Param(model.T, model.F, initialize=D, default=0)
model.D_beverage = pyo.Param(model.T, model.B, initialize=D, default=0)
model.R_food = pyo.Param(model.T, model.F, initialize=R, default=0)
model.R_beverage = pyo.Param(model.T, model.B, initialize=R, default=0)
model.C_food = pyo.Param(model.F, initialize=C, default=0)
model.C_beverage = pyo.Param(model.B, initialize=C, default=0)
model.P_food = pyo.Param(model.F, initialize=P, default=0)
model.P_beverage = pyo.Param(model.B, initialize=P, default=0)

# Decision Variables
model.X_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.X_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)
model.S_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.S_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)
model.Sold_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.Sold_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)

# Objective Function
def total_profit(model):
    return sum(
        (model.R_food[t, f] * model.Sold_food[t, f] - 
         model.FC_food[t, f] * model.X_food[t, f] - 
         model.VC_food[t, f] * model.X_food[t, f] - 
         model.SC_food[t, f] * model.S_food[t, f]) + 
        (model.R_beverage[t, b] * model.Sold_beverage[t, b] - 
         model.FC_beverage[t, b] * model.X_beverage[t, b] - 
         model.VC_beverage[t, b] * model.X_beverage[t, b] - 
         model.SC_beverage[t, b] * model.S_beverage[t, b]) 
        for t in model.T for f in model.F for b in model.B
    )
model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# 1. Production Capacity
def production_capacity_food(model, t, f):
    return model.X_food[t, f] <= model.P_food[f]
model.production_capacity_food_constraint = pyo.Constraint(model.T, model.F, rule=production_capacity_food)

def production_capacity_beverage(model, t, b):
    return model.X_beverage[t, b] <= model.P_beverage[b]
model.production_capacity_beverage_constraint = pyo.Constraint(model.T, model.B, rule=production_capacity_beverage)

# 2. Storage Capacity
def storage_capacity_food(model, t, f):
    return model.S_food[t, f] <= model.C_food[f]
model.storage_capacity_food_constraint = pyo.Constraint(model.T, model.F, rule=storage_capacity_food)

def storage_capacity_beverage(model, t, b):
    return model.S_beverage[t, b] <= model.C_beverage[b]
model.storage_capacity_beverage_constraint = pyo.Constraint(model.T, model.B, rule=storage_capacity_beverage)

# 3. Demand Fulfillment
def demand_fulfillment_food(model, t, f):
    return model.Sold_food[t, f] <= model.D_food[t, f]
model.demand_fulfillment_food_constraint = pyo.Constraint(model.T, model.F, rule=demand_fulfillment_food)

def demand_fulfillment_beverage(model, t, b):
    return model.Sold_beverage[t, b] <= model.D_beverage[t, b]
model.demand_fulfillment_beverage_constraint = pyo.Constraint(model.T, model.B, rule=demand_fulfillment_beverage)

# 4. Inventory Balance
def inventory_balance_food(model, t, f):
    if t == 1:
        return model.S_food[t, f] == model.X_food[t, f] - model.Sold_food[t, f] 
    else:
        return model.S_food[t, f] == model.S_food[t-1, f] + model.X_food[t, f] - model.Sold_food[t, f]
model.inventory_balance_food_constraint = pyo.Constraint(model.T, model.F, rule=inventory_balance_food)

def inventory_balance_beverage(model, t, b):
    if t == 1:
        return model.S_beverage[t, b] == model.X_beverage[t, b] - model.Sold_beverage[t, b] 
    else:
        return model.S_beverage[t, b] == model.S_beverage[t-1, b] + model.X_beverage[t, b] - model.Sold_beverage[t, b]
model.inventory_balance_beverage_constraint = pyo.Constraint(model.T, model.B, rule=inventory_balance_beverage)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Total Profit:", pyo.value(model.profit)) 

# Access specific decision variable values
# Example:
# print("Quantity of Food 1 produced in period 1:", pyo.value(model.X_food[1, 1]))
```

**Explanation:**

1.  **Initialization:** The code starts by importing the `pyomo` library and defining sample data for the sets and parameters of your problem. Replace the sample data with your actual data. 
2.  **Model Creation:** A concrete model is created using `pyo.ConcreteModel()`. 
3.  **Sets and Parameters:** Sets are defined for time periods (T), food products (F), and beverage products (B) using `pyo.Set()`. Parameters like costs, demand, revenue, and capacities are defined using `pyo.Param()`.
4.  **Decision Variables:** The decision variables representing production, storage, and sales are created using `pyo.Var()`. The `within` argument specifies the domain of the variables (e.g., non-negative integers, non-negative reals).
5.  **Objective Function:** The `total_profit` function calculates the total profit based on the decision variables. The `pyo.Objective()` function sets this as the objective to be maximized.
6.  **Constraints:**  Each constraint is defined as a separate Python function, and `pyo.Constraint()` is used to add them to the model. The functions ensure that production, storage, demand, and inventory balance constraints are met.
7.  **Solving the Model:** A solver (in this case, 'cbc') is selected using `pyo.SolverFactory()`, and the model is solved.
8.  **Output:** The results and the total profit are printed. You can access and print specific decision variable values using `pyo.value()`.

Remember to install the necessary packages (Pyomo and a solver like CBC) before running this code. You can usually install them using `pip install pyomo` and `pip install cbc`.


## 4. Input problem data and try running the generated code

In [467]:
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
time_periods = [1,2,3,4,5,6,7,8,9,10,11,12]
food_products = [1, 2]
beverage_products = [1, 2, 3]

#DATA LOADING 
fixed_cost_production = pd.read_csv("fixed_cost_production.csv", index_col=[0])
variable_cost_production = pd.read_csv("variable_cost_production.csv", index_col=[0])
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", index_col=[0])
demand = pd.read_csv("demand.csv", index_col=[0])
revenue = pd.read_csv("revenue.csv", index_col=[0])



# Function to convert DataFrame to the desired dictionary format
def convert_to_dict(df, item_type):
    if item_type == "food":
        data = df.iloc[:2, :]  # Select first two rows for food
    elif item_type == "beverage":
        data = df.iloc[2:, :]  # Select last three rows for beverage

    result_dict = {(i+1, t+1): data.iloc[i, t] for i in range(data.shape[0]) for t in range(data.shape[1])}
    return result_dict

# Convert each DataFrame to the desired dictionary format
fixed_cost_production_food = convert_to_dict(fixed_cost_production, "food")
fixed_cost_production_beverage = convert_to_dict(fixed_cost_production, "beverage")

variable_cost_production_food = convert_to_dict(variable_cost_production, "food")
variable_cost_production_beverage = convert_to_dict(variable_cost_production, "beverage")

variable_cost_storage_food = convert_to_dict(variable_cost_storage, "food")
variable_cost_storage_beverage = convert_to_dict(variable_cost_storage, "beverage")

demand_food = convert_to_dict(demand, "food")
demand_beverage = convert_to_dict(demand, "beverage")

revenue_food = convert_to_dict(revenue, "food")
revenue_beverage = convert_to_dict(revenue, "beverage")
###END


# Create a Concrete Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=time_periods)
model.F = pyo.Set(initialize=food_products)
model.B = pyo.Set(initialize=beverage_products)

# Parameters
model.FC_food = pyo.Param( model.F,model.T, initialize=fixed_cost_production_food, default=0)
model.FC_beverage = pyo.Param(model.B,model.T, initialize=fixed_cost_production_beverage, default=0)
model.VC_food = pyo.Param( model.F,model.T, initialize=variable_cost_production_food, default=0)
model.VC_beverage = pyo.Param(model.B,model.T,  initialize=variable_cost_production_beverage, default=0)
model.SC_food = pyo.Param( model.F,model.T, initialize=variable_cost_storage_food, default=0)
model.SC_beverage = pyo.Param(model.B,model.T,  initialize=variable_cost_storage_beverage, default=0)
model.D_food = pyo.Param(model.F,model.T,  initialize=demand_food, default=0)
model.D_beverage = pyo.Param( model.B, model.T,initialize=demand_beverage, default=0)
model.R_food = pyo.Param( model.F,model.T, initialize=revenue_food, default=0)
model.R_beverage = pyo.Param(model.B,model.T,  initialize=revenue_beverage, default=0)
model.C_food = pyo.Param(model.F, initialize=C, default=0)
model.C_beverage = pyo.Param(model.B, initialize=C, default=0)
model.P_food = pyo.Param(model.F, initialize=P, default=0)
model.P_beverage = pyo.Param(model.B, initialize=P, default=0)

# Decision Variables
model.X_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.X_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)
model.S_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.S_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)
model.Sold_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.Sold_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)

# Objective Function
def total_profit(model):
    return sum(
        (model.R_food[t, f] * model.Sold_food[t, f] - 
         model.FC_food[t, f] * model.X_food[t, f] - 
         model.VC_food[t, f] * model.X_food[t, f] - 
         model.SC_food[t, f] * model.S_food[t, f]) + 
        (model.R_beverage[t, b] * model.Sold_beverage[t, b] - 
         model.FC_beverage[t, b] * model.X_beverage[t, b] - 
         model.VC_beverage[t, b] * model.X_beverage[t, b] - 
         model.SC_beverage[t, b] * model.S_beverage[t, b]) 
        for t in model.T for f in model.F for b in model.B
    )
model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# 1. Production Capacity
def production_capacity_food(model, t, f):
    return model.X_food[t, f] <= model.P_food[f]
model.production_capacity_food_constraint = pyo.Constraint(model.T, model.F, rule=production_capacity_food)

def production_capacity_beverage(model, t, b):
    return model.X_beverage[t, b] <= model.P_beverage[b]
model.production_capacity_beverage_constraint = pyo.Constraint(model.T, model.B, rule=production_capacity_beverage)

# 2. Storage Capacity
def storage_capacity_food(model, t, f):
    return model.S_food[t, f] <= model.C_food[f]
model.storage_capacity_food_constraint = pyo.Constraint(model.T, model.F, rule=storage_capacity_food)

def storage_capacity_beverage(model, t, b):
    return model.S_beverage[t, b] <= model.C_beverage[b]
model.storage_capacity_beverage_constraint = pyo.Constraint(model.T, model.B, rule=storage_capacity_beverage)

# 3. Demand Fulfillment
def demand_fulfillment_food(model, t, f):
    return model.Sold_food[t, f] <= model.D_food[t, f]
model.demand_fulfillment_food_constraint = pyo.Constraint(model.T, model.F, rule=demand_fulfillment_food)

def demand_fulfillment_beverage(model, t, b):
    return model.Sold_beverage[t, b] <= model.D_beverage[t, b]
model.demand_fulfillment_beverage_constraint = pyo.Constraint(model.T, model.B, rule=demand_fulfillment_beverage)

# 4. Inventory Balance
def inventory_balance_food(model, t, f):
    if t == 1:
        return model.S_food[t, f] == model.X_food[t, f] - model.Sold_food[t, f] 
    else:
        return model.S_food[t, f] == model.S_food[t-1, f] + model.X_food[t, f] - model.Sold_food[t, f]
model.inventory_balance_food_constraint = pyo.Constraint(model.T, model.F, rule=inventory_balance_food)

def inventory_balance_beverage(model, t, b):
    if t == 1:
        return model.S_beverage[t, b] == model.X_beverage[t, b] - model.Sold_beverage[t, b] 
    else:
        return model.S_beverage[t, b] == model.S_beverage[t-1, b] + model.X_beverage[t, b] - model.Sold_beverage[t, b]
model.inventory_balance_beverage_constraint = pyo.Constraint(model.T, model.B, rule=inventory_balance_beverage)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Total Profit:", pyo.value(model.profit)) 

ERROR: Rule failed when generating expression for Objective profit with index
None: KeyError: "Index '(3, 1)' is not valid for indexed component 'R_food'"
ERROR: Constructing component 'profit' from data=None failed:
        KeyError: "Index '(3, 1)' is not valid for indexed component 'R_food'"


KeyError: "Index '(3, 1)' is not valid for indexed component 'R_food'"

## 5. Correct the code to verify model viability (optional)

In [476]:
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
time_periods = [1,2,3,4,5,6,7,8,9,10,11,12]
food_products = [1, 2]
beverage_products = [1, 2, 3]

#DATA LOADING 
fixed_cost_production = pd.read_csv("fixed_cost_production.csv", index_col=[0])
variable_cost_production = pd.read_csv("variable_cost_production.csv", index_col=[0])
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", index_col=[0])
demand = pd.read_csv("demand.csv", index_col=[0])
revenue = pd.read_csv("revenue.csv", index_col=[0])



# Function to convert DataFrame to the desired dictionary format
def convert_to_dict(df, item_type):
    if item_type == "food":
        data = df.iloc[:2, :]  # Select first two rows for food
    elif item_type == "beverage":
        data = df.iloc[2:, :]  # Select last three rows for beverage

    result_dict = {(t+1, i+1): data.iloc[i, t] for i in range(data.shape[0]) for t in range(data.shape[1])}
    return result_dict

# Convert each DataFrame to the desired dictionary format
fixed_cost_production_food = convert_to_dict(fixed_cost_production, "food")
fixed_cost_production_beverage = convert_to_dict(fixed_cost_production, "beverage")

variable_cost_production_food = convert_to_dict(variable_cost_production, "food")
variable_cost_production_beverage = convert_to_dict(variable_cost_production, "beverage")

variable_cost_storage_food = convert_to_dict(variable_cost_storage, "food")
variable_cost_storage_beverage = convert_to_dict(variable_cost_storage, "beverage")

demand_food = convert_to_dict(demand, "food")
demand_beverage = convert_to_dict(demand, "beverage")

revenue_food = convert_to_dict(revenue, "food")
revenue_beverage = convert_to_dict(revenue, "beverage")
###END
C_f = {1: 580, 2: 687}  
C_b = {1:599, 2:788, 3:294}
P_f = {1: 1080, 2: 908} 
P_b = {1: 408, 2: 1000,3:403} 

# Create a Concrete Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=time_periods)
model.F = pyo.Set(initialize=food_products)
model.B = pyo.Set(initialize=beverage_products)

# Parameters
model.FC_food = pyo.Param( model.T,model.F, initialize=fixed_cost_production_food, default=0)
model.FC_beverage = pyo.Param(model.T, model.B,initialize=fixed_cost_production_beverage, default=0)
model.VC_food = pyo.Param( model.T,model.F, initialize=variable_cost_production_food, default=0)
model.VC_beverage = pyo.Param(model.T,model.B,  initialize=variable_cost_production_beverage, default=0)
model.SC_food = pyo.Param( model.T,model.F, initialize=variable_cost_storage_food, default=0)
model.SC_beverage = pyo.Param(model.T,model.B,  initialize=variable_cost_storage_beverage, default=0)
model.D_food = pyo.Param(model.T, model.F, initialize=demand_food, default=0)
model.D_beverage = pyo.Param( model.T, model.B,initialize=demand_beverage, default=0)
model.R_food = pyo.Param(model.T, model.F, initialize=revenue_food, default=0)
model.R_beverage = pyo.Param(model.T,model.B,  initialize=revenue_beverage, default=0)
model.C_food = pyo.Param(model.F, initialize=C_f, default=0)
model.C_beverage = pyo.Param(model.B, initialize=C_b, default=0)
model.P_food = pyo.Param(model.F, initialize=P_f, default=0)
model.P_beverage = pyo.Param(model.B, initialize=P_b, default=0)

# Decision Variables
model.X_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.X_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)
model.S_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.S_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)
model.Sold_food = pyo.Var(model.T, model.F, within=pyo.NonNegativeIntegers)
model.Sold_beverage = pyo.Var(model.T, model.B, within=pyo.NonNegativeReals)

# Objective Function
def total_profit(model):
    prof = (
        sum(
            model.R_food[t, f] * model.Sold_food[t, f] - 
            model.FC_food[t, f] * model.X_food[t, f] - 
            model.VC_food[t, f] * model.X_food[t, f] - 
            model.SC_food[t, f] * model.S_food[t, f] 
            for t in model.T for f in model.F
        ) + 
        sum(
            model.R_beverage[t, b] * model.Sold_beverage[t, b] - 
            model.FC_beverage[t,b] * model.X_beverage[t, b] - 
            model.VC_beverage[t, b] * model.X_beverage[t, b] - 
            model.SC_beverage[t, b] * model.S_beverage[t, b] 
            for t in model.T for b in model.B
        )
    )
    return prof
model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# 1. Production Capacity
def production_capacity_food(model, t, f):
    return model.X_food[t, f] <= model.P_food[f]
model.production_capacity_food_constraint = pyo.Constraint(model.T, model.F, rule=production_capacity_food)

def production_capacity_beverage(model, t, b):
    return model.X_beverage[t, b] <= model.P_beverage[b]
model.production_capacity_beverage_constraint = pyo.Constraint(model.T, model.B, rule=production_capacity_beverage)

# 2. Storage Capacity
def storage_capacity_food(model, t, f):
    return model.S_food[t, f] <= model.C_food[f]
model.storage_capacity_food_constraint = pyo.Constraint(model.T, model.F, rule=storage_capacity_food)

def storage_capacity_beverage(model, t, b):
    return model.S_beverage[t, b] <= model.C_beverage[b]
model.storage_capacity_beverage_constraint = pyo.Constraint(model.T, model.B, rule=storage_capacity_beverage)

# 3. Demand Fulfillment
def demand_fulfillment_food(model, t, f):
    return model.Sold_food[t, f] <= model.D_food[t, f]
model.demand_fulfillment_food_constraint = pyo.Constraint(model.T, model.F, rule=demand_fulfillment_food)

def demand_fulfillment_beverage(model, t, b):
    return model.Sold_beverage[t, b] <= model.D_beverage[t, b]
model.demand_fulfillment_beverage_constraint = pyo.Constraint(model.T, model.B, rule=demand_fulfillment_beverage)

# 4. Inventory Balance
def inventory_balance_food(model, t, f):
    if t == 1:
        return model.S_food[t, f] == model.X_food[t, f] - model.Sold_food[t, f] 
    else:
        return model.S_food[t, f] == model.S_food[t-1, f] + model.X_food[t, f] - model.Sold_food[t, f]
model.inventory_balance_food_constraint = pyo.Constraint(model.T, model.F, rule=inventory_balance_food)

def inventory_balance_beverage(model, t, b):
    if t == 1:
        return model.S_beverage[t, b] == model.X_beverage[t, b] - model.Sold_beverage[t, b] 
    else:
        return model.S_beverage[t, b] == model.S_beverage[t-1, b] + model.X_beverage[t, b] - model.Sold_beverage[t, b]
model.inventory_balance_beverage_constraint = pyo.Constraint(model.T, model.B, rule=inventory_balance_beverage)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Total Profit:", pyo.value(model.profit)) 


Problem: 
- Name: unknown
  Lower bound: 508.1906
  Upper bound: 508.1906
  Number of objectives: 1
  Number of constraints: 240
  Number of variables: 180
  Number of nonzeros: 415
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.03982186317443848
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Total Profit: 508.1906


## 6. Print the responses

In [477]:
print(response.text)

## Indices:

*  $t$: Time period, $t = 1, 2, ..., 12$
*  $f$: Food product,  $f = 1, 2$
*  $b$: Beverage product, $b = 1, 2, 3$

## Parameters:

*  $FC_{t,f}$: Fixed cost of producing one unit of food $f$ in period $t$ 
*  $FC_{t,b}$: Fixed cost of producing one unit of beverage $b$ in period $t$ 
*  $VC_{t,f}$: Variable cost of producing one unit of food  $f$ in period $t$
*  $VC_{t,b}$: Variable cost of producing one liter of beverage $b$ in period $t$ 
*  $SC_{t,f}$: Variable cost of storing one unit of food $f$ in period $t$ 
*  $SC_{t,b}$: Variable cost of storing one liter of beverage $b$ in period $t$ 
*  $D_{t,f}$: Demand for food $f$ in period $t$ 
*  $D_{t,b}$: Demand for beverage $b$ in period $t$ (in liters)
*  $R_{t,f}$: Revenue for selling one unit of food $f$ in period $t$
*  $R_{t,b}$: Revenue for selling one liter of beverage $b$ in period $t$
*  $C_{f}$: Storage capacity for food $f$ (in units) 
*  $C_{b}$: Storage capacity for beverage $b$ (in liters)
*  $P_{f}$: Pro

In [478]:
print(response2.text)

## Objective Function:

Maximize the total profit over all time periods:

$\sum_{t=1}^{12} \left( \sum_{f=1}^{2} (R_{t,f} \cdot Sold_{t,f} - FC_{t,f} \cdot X_{t,f}  - VC_{t,f} \cdot X_{t,f} - SC_{t,f} \cdot S_{t,f})  + \sum_{b=1}^{3} (R_{t,b} \cdot Sold_{t,b} - FC_{t,b} \cdot X_{t,b} - VC_{t,b} \cdot X_{t,b} - SC_{t,b} \cdot S_{t,b}) \right)$ 



In [479]:
print(response3.text)

## Constraints:

**1. Production Capacity:**

*   $X_{t,f} \leq P_f$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $X_{t,b} \leq P_b$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$

**2. Storage Capacity:**

*   $S_{t,f} \leq C_f$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $S_{t,b} \leq C_b$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$

**3. Demand Fulfillment:**

*   $Sold_{t,f} \leq D_{t,f}$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $Sold_{t,b} \leq D_{t,b}$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$

**4. Inventory Balance:**

*   $S_{t,f} = S_{t-1,f} + X_{t,f} - Sold_{t,f}$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$ 
*   $S_{t,b} = S_{t-1,b} + X_{t,b} - Sold_{t,b}$ for all $t = 1, 2, ..., 12$ and $b = 1, 2, 3$
*   $S_{0,f} = 0$ for all $f = 1, 2$ (assuming no initial inventory)
*   $S_{0,b} = 0$ for all $b = 1, 2, 3$ (assuming no initial inventory)

**5. Non-Negativity:**

*   $X_{t,f} \geq 0$ for all $t = 1, 2, ..., 12$ and $f = 1, 2$
*   $X_{t,b} \geq 0$ for all $t =

In [480]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
time_periods = [t+1 for t in range(12)]  
food_products = [1, 2]
beverage_products = [1, 2, 3]

# Parameters (Sample Data)
FC = {(1, 1): 10, (1, 2): 12, (2, 1): 11, (2, 2): 13} # etc. for all t, f
VC = {(1, 1): 5, (1, 2): 6, (2, 1): 5.5, (2, 2): 6.5} # etc. for all t, f
SC = {(1, 1): 2, (1, 2): 2.5, (2, 1): 2.2, (2, 2): 2.7} # etc. for all t, f
D = {(1, 1): 100, (1, 2): 120, (2, 1): 110, (2, 2): 130} # etc. for all t, f
R = {(1, 1): 20, (1, 2): 22, (2, 1): 21, (2, 2): 23} # etc. for all t, f
C = {1: 200, 2: 250}  
P = {1: 150, 2: 180} 

# Create a Concrete Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=time_periods)
model.F = pyo.Set(initialize=food_products)
model.B = pyo.Set(initialize=beverage_products)

# Parameters
model.FC_food = pyo.Param(model.T, model.F, initialize=FC, default=0)
model.FC_beverage = pyo.Param(model.T, model.B, initialize=FC, default=0)
model.VC_food = p